In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

def generate_forex_data(start_price, start_datetime, end_datetime, max_price, min_price):
    # Initialize the datetime
    current_datetime = datetime.strptime(start_datetime, '%d/%m/%Y %H:%M')

    # Calculate the number of entries based on the time difference
    time_diff = end_datetime - current_datetime
    num_entries = int(time_diff.total_seconds() / 60)  # Convert to minutes

    # Initialize data lists
    datetimes = []
    opens = []
    highs = []
    lows = []
    closes = []
    orders = []  # Buy (1) or Sell (0) orders
    thresholds = []
    volumes = []

    # Generate data
    price = start_price
    for _ in range(num_entries):
        # Random walk for Open, High, Low, Close prices
        open_ = price + np.random.uniform(-0.0005, 0.0005)
        high = min(price + np.random.uniform(0, 0.0005), max_price)
        low = max(price - np.random.uniform(0, 0.0005), min_price)

        # Randomly decide the candle direction (up or down)
        if np.random.choice([True, False]):
            close = min(max(low + np.random.uniform(0, 0.0005), min_price), max_price)
            order = 1  # Buy order
        else:
            close = max(min(high - np.random.uniform(0, 0.0005), max_price), min_price)
            order = 0  # Sell order

        # Random candle volume
        volume = np.random.randint(100, 1000)  # Random volume between 100 and 1000

        signal = np.random.choice([0, 1])  # Random signal
        threshold = high - low  # Threshold is the range

        # Append to lists
        datetimes.append(current_datetime)
        opens.append(open_)
        highs.append(high)
        lows.append(low)
        closes.append(close)
        orders.append(order)
        thresholds.append(threshold)

        # Update for next iteration
        price = close
        current_datetime += timedelta(minutes=1)  # Increment time by one minute

    # Create DataFrame
    forex_data = pd.DataFrame({
        'Datetime': datetimes,
        'Open': opens,
        'High': highs,
        'Low': lows,
        'Close': closes,
        'Order': orders,
        'Threshold': thresholds,
    })

    # Format the 'Datetime' column
    forex_data['Datetime'] = forex_data['Datetime'].dt.strftime('%d/%m/%Y %H:%M')

    return forex_data

# Parameters to generate data
start_price = 1.06279
start_datetime = '1/1/2022 00:00'
end_datetime = datetime.now()  # Get the current date and time
max_price = 1.127575
min_price = 0.953545

# Generate the data
forex_data = generate_forex_data(start_price, start_datetime, end_datetime, max_price, min_price)

# Define the file path on Google Drive
file_path = '/content/drive/My Drive/Data.csv'

# Save the DataFrame to a CSV file
forex_data.to_csv(file_path, index=False)

# Display a plot for the last 1000 rows of the generated data
last_1000_rows = forex_data.tail(100)
plt.figure(figsize=(12, 6))
plt.plot(last_1000_rows['Datetime'], last_1000_rows['Open'], label='Open Price', color='green')
plt.plot(last_1000_rows['Datetime'], last_1000_rows['High'], label='High Price', color='red')
plt.plot(last_1000_rows['Datetime'], last_1000_rows['Low'], label='Low Price', color='blue')
plt.plot(last_1000_rows['Datetime'], last_1000_rows['Close'], label='Close Price', color='purple')
plt.title('Forex Prices - Last 1000 Rows')
plt.xlabel('Datetime')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()
